In [17]:
import openrouteservice as ors
import folium

client = ors.Client(key='5b3ce3597851110001cf62480413a152b510459cabf5e54bcfb4a7a9')
#client = ors.Client(key='5b3ce3597851110001cf6248f4b857eea0554d93886b4c09e748b2bf')

In [2]:
import re
import pandas as pd
import math

import numpy as np

from operator import itemgetter
from heapq import nsmallest

from itertools import chain

import time

In [10]:
df_property = pd.read_csv('../data/curated/property.csv')

property_coor = []
for index, row in df_property.iterrows():
    lat = float(re.findall(r'\[(-\d+.\d+)',str(row['coordinates']))[0])
    long = float(re.findall(r'(\d+.\d+)\]',str(row['coordinates']))[0])
    property_coor.append((long,lat))

print(len(property_coor))

# set the location of Melbourne CBD
cbd_coor = [(144.946457,-37.840935)]


15222


In [14]:

iter = 0
combine_coor = property_coor[iter*3500:iter*3500+3500] + cbd_coor

print(combine_coor[len(combine_coor)-1])
print(len(combine_coor))


(144.946457, -37.840935)
3501


In [18]:
def get_cbd_ors(iter):
    
    combine_coor = property_coor[iter*3500:iter*3500+3500] + cbd_coor

    matrix = client.distance_matrix(
        locations=combine_coor,
        profile='driving-car',
        metrics=['distance', 'duration'],
        validate=False,
        destinations=[len(combine_coor)-1],
        sources=list(range(len(combine_coor)-1)),
    )


    print("Durations in secs: {}\n".format(matrix['durations']))
    print("Distances in m: {}".format(matrix['distances']))

    # we only want to call the whole function once at each main iteration
    result_list = []
    result_list.append(matrix['durations'])
    result_list.append(matrix['distances'])
    return result_list

# A total of 3500 routes is allowed for each call of api, so that we can pass 3500 properties at each time
# with corresponding 3 closest schools at each time

duration_list = []
distance_list = []

num_call = len(df_property)//3500 + 1

for i in range(num_call):

    result = get_cbd_ors(i)

    duration_list.append(result[0])
    distance_list.append(result[1])


print(duration_list)
print(distance_list)
